In [1]:
import json
from tqdm import tqdm
import k3d
import numpy as np
from matplotlib import pyplot as plt

In [2]:
with open('/scratches/octopus_2/fml35/datasets/scannet/scan2cad_annotations/full_annotations.json','r') as f:
    annotations = json.load(f)

In [3]:
print(len(annotations))

1506


In [4]:
def get_top8_classes_scannet():                                                                                                                                                                                                                                                                                           
    top = {}
    top["03211117"] = "display"
    top["04379243"] = "table"
    top["02808440"] = "bathtub"
    top["02747177"] = "bin"
    top["04256520"] = "sofa"
    top["03001627"] = "chair"
    top["02933112"] = "cabinet"
    top["02871439"] = "bookshelf"
    top["02818832"] = "bed"
    return top

cat_id_to_label = get_top8_classes_scannet()

In [5]:
with open('/scratches/octopus_2/fml35/datasets/scannet/data_splits/scannetv2_train.txt','r') as f:
    train = f.readlines()
train_scenes = [x.strip() for x in train]

with open('/scratches/octopus_2/fml35/datasets/scannet/data_splits/scannetv2_val.txt','r') as f:
    val = f.readlines()
val_scenes = [x.strip() for x in val]

In [22]:
counter_bad_annos = 0
data = {}
data["train"] = {}
data["val"] = {}
for kind in data:
    data[kind]['scale'] = []
    data[kind]['bbox'] = []
    data[kind]['center'] = []
    data[kind]['category'] = []
    data[kind]['model_id'] = []
    data[kind]['sym'] = []

for i in tqdm(range(len(annotations))):
    scene = annotations[i]['id_scan']
    kind = None
    if scene in train_scenes:
        kind = 'train'
    elif scene in val_scenes:
        kind = 'val'

    for j in range(len(annotations[i]['aligned_models'])):
        catid = annotations[i]['aligned_models'][j]['catid_cad']
        if catid in cat_id_to_label:
            data[kind]['scale'].append(annotations[i]['aligned_models'][j]['trs']['scale'])
            data[kind]['bbox'].append(annotations[i]['aligned_models'][j]['bbox'])
            data[kind]['center'].append(annotations[i]['aligned_models'][j]['center'])
            data[kind]['sym'].append(annotations[i]['aligned_models'][j]['sym'])
            data[kind]['model_id'].append(annotations[i]['aligned_models'][j]['id_cad'])
            data[kind]['category'].append(cat_id_to_label[catid])

            if np.any(np.array(annotations[i]['aligned_models'][j]['trs']['scale']) < 0.01):
                print(annotations[i]['aligned_models'][j]['trs']['scale'])
                print(annotations[i]['aligned_models'][j]['bbox'])
                print(annotations[i]['aligned_models'][j]['center'])
                print('scene',scene)
                counter_bad_annos += 1
    
print('counter_bad_annos',counter_bad_annos)
    

100%|██████████| 1506/1506 [00:00<00:00, 7560.33it/s]

[0.7699015223851816, 0.7896077849998121, 7.924078834127581e-13]
[0.36867451667785645, 0.29054951667785645, 0.17222100496292114]
[0.04869849979877472, 0.07770050317049026, -0.024903997778892517]
scene scene0263_01
[0.698484173823658, 0.7310481477929225, 8.046704945211398e-12]
[0.42902350425720215, 0.2492540031671524, 0.06173650175333023]
[-0.032790496945381165, -0.01833800971508026, -0.004385501146316528]
scene scene0010_01
[0.7632742943173566, 0.8413592126031525, 2.0009225452695845e-13]
[0.3894835114479065, 0.2797204852104187, 0.14163050055503845]
[0.045743510127067566, 0.12692050635814667, 0.013335496187210083]
scene scene0010_01
[0.7161254327426432, 0.600890581693934, 2.7790540779826833e-11]
[0.4128339886665344, 0.2817454934120178, 0.013699499890208244]
[-0.02493099868297577, -0.24410748854279518, -0.008322499692440033]
scene scene0010_01
[1.16195629762197, 1.7142671847182693, 6.6237974314335146e-12]
[0.4640580117702484, 0.18562349677085876, 0.013921499252319336]
[-0.0010460019111633

In [7]:
for kind in data:
    for key in data[kind]:
        data[kind][key] = np.array(data[kind][key])
        assert len(data[kind][key]) == len(data[kind]['category'])

In [8]:
print(data['train']['scale'].shape)
print(data['train']['bbox'].shape)

for key in data['train']:
    print(key)

(9894, 3)
(9894, 3)
scale
bbox
center
category
model_id
sym


In [19]:
train_scale_by_model_id  = {}
train_scale_by_category = {}
for i in range(len(data['train']['scale'])):
    model_id = data['train']['model_id'][i]
    if model_id not in train_scale_by_model_id:
        train_scale_by_model_id[model_id] = []
    train_scale_by_model_id[model_id].append(data['train']['scale'][i])

    category = data['train']['category'][i]
    if category not in train_scale_by_category:
        train_scale_by_category[category] = []
    train_scale_by_category[category].append(data['train']['scale'][i])

print('len(train_scale_by_model_id)',len(train_scale_by_model_id))


for key in train_scale_by_model_id:
    train_scale_by_model_id[key] = np.array(train_scale_by_model_id[key])
for key in train_scale_by_category:
    train_scale_by_category[key] = np.array(train_scale_by_category[key])

# print(train_scale_by_model_id['d0959256c79f60872a9c9a7e9520eea'])

median_scale_by_model_id = {}
for key in train_scale_by_model_id:
    median_scale_by_model_id[key] = np.median(train_scale_by_model_id[key],axis=0).tolist()
median_scale_by_category = {}
for key in train_scale_by_category:
    median_scale_by_category[key] = np.median(train_scale_by_category[key],axis=0).tolist()

print(len(median_scale_by_model_id))
for i in range(len(data['val']['model_id'])):
    model_id = data['val']['model_id'][i]
    if model_id not in median_scale_by_model_id:
        median_scale_by_model_id[model_id] = median_scale_by_category[data['val']['category'][i]]
print(len(median_scale_by_model_id))

for key in median_scale_by_model_id:
    if np.any(np.array(median_scale_by_model_id[key]) < 0.1) or np.any(np.array(median_scale_by_model_id[key]) > 7):
        print(key,median_scale_by_model_id[key])
# with open('/scratch/fml35/datasets/own_datasets/leveraging_geometry_for_shape_estimation/scannet/scaling/dfdf_median_scale_by_model_id_category_median_if_not_in_train.json','w') as f:
#     json.dump(median_scale_by_model_id,f)

# print(len(train_scale_by_model_id))
# counter = 0
# for key in train_scale_by_model_id:
#     print(key, len(train_scale_by_model_id[key]),train_scale_by_model_id[key])
#     counter += 1
#     if counter == 5:
#         break

# all_lengths = []
# for key in train_scale_by_model_id:
#     all_lengths.append(len(train_scale_by_model_id[key]))

# print('mean lenght', np.mean(all_lengths))
# print('median lenght', np.median(all_lengths))


len(train_scale_by_model_id) 2283
2283
2589
4270cebd8ca0f2593dfb5aeeb86be115 [0.7699015223851816, 0.7896077849998121, 7.924078834127581e-13]
b2ec76ac8a5c7afa40ff33d80c1d7d1e [0.698484173823658, 0.7310481477929225, 8.046704945211398e-12]
2ba6e0c4ea459f84f5f79806bd65844f [0.7161254327426432, 0.600890581693934, 2.7790540779826833e-11]
d9bac9c418667d2ba62bc668cb34e698 [0.7407739272147467, 0.5826065149401614, 1.3349657352308962e-11]
d0959256c79f60872a9c9a7e9520eea [0.7280809315962707, 0.7593965749871695, 8.782145326494881e-12]
28b6e44e37586fd797e8123bbbc761d8 [1.1084516133937181, 1.1324651978405809, 1.641563164078537e-11]
1707d858ffaf146e640578ae55230ebc [0.8350898517806699, 0.9302174293633905, 4.758778324705272e-13]
c16f53ea58244af7d5297776dd9015b5 [0.6122090035517788, 0.549088577866, 6.076462951691035e-13]
d7b87d0083bf5568fd28950562697757 [0.649284667758558, 0.626248560495938, 4.796513708851013e-12]
c11449b64486b4df4240b2500b599345 [1.4506032946578975, 1.4571774971791482, 7.46370111490054

In [10]:


N_in_train = 0
N_offset_in_train_correct = 0
N_not_in_train = 0
N_offset_not_in_train_correct = 0
print("len(data['val']['scale'])",len(data['val']['scale']))
for i in range(len(data['val']['scale'])):
    scale_val = data['val']['scale'][i]
    model_id = data['val']['model_id'][i]
    if model_id in median_scale_by_model_id:
        N_in_train += 1
        scale_median_train = median_scale_by_model_id[model_id]
        offset_s = np.mean(np.abs(scale_val / scale_median_train - 1))
        if offset_s < 0.2:
            N_offset_in_train_correct += 1
    else:
        print(model_id)
        N_not_in_train += 1
        scale_median_train = median_scale_by_category[data['val']['category'][i]]

        offset_s = np.mean(np.abs(scale_val / scale_median_train - 1))
        if offset_s < 0.2:
            N_offset_not_in_train_correct += 1


print('N_in_train ',N_in_train)
print('N_offset_in_train_correct ',N_offset_in_train_correct)
print('N_not_in_train ',N_not_in_train)
print('N_offset_not_in_train_correct ',N_offset_not_in_train_correct)

len(data['val']['scale']) 2844
N_in_train  2844
N_offset_in_train_correct  2040
N_not_in_train  0
N_offset_not_in_train_correct  0


In [11]:
len(data['val']['scale']) 2844
N_in_train  2332
N_offset_in_train_correct  1778
N_not_in_train  512
N_offset_not_in_train_correct  262



SyntaxError: invalid syntax (3328206143.py, line 1)